<a href="https://colab.research.google.com/github/ValerieLangat/DS-Project-2---Predictive-Modeling/blob/master/dialing_back_as_simple_as_possible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install category_encoders

In [0]:
import category_encoders as ce
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred)) 

In [0]:
data = pd.read_csv('https://raw.githubusercontent.com/ValerieLangat/DS-Project-2---Predictive-Modeling/master/efw_cc.csv')
data = data.drop(["ISO_code", "rank", "quartile"], axis=1)
data = data.dropna(thresh=17)
data = data.fillna(data.median())

In [0]:
data['year'] = pd.to_datetime(data['year'])
data['year'] = data['year'].map(lambda x: x.year)
data['id'] = data['year']

In [0]:
data['id'] = data[['year', 'countries']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)

In [0]:
data.rename(columns = {'ECONOMIC FREEDOM':'ECONOMIC_FREEDOM'}, inplace = True)

In [60]:
data

,year,countries,ECONOMIC_FREEDOM,1a_government_consumption,1b_transfers,1c_gov_enterprises,1d_top_marg_tax_rate,1_size_government,2a_judicial_independence,2b_impartial_courts,2c_protection_property_rights,2d_military_interference,2e_integrity_legal_system,2f_legal_enforcement_contracts,2g_restrictions_sale_real_property,2h_reliability_police,2i_business_costs_crime,2j_gender_adjustment,2_property_rights,3a_money_growth,3b_std_inflation,3c_inflation,3d_freedom_own_foreign_currency,3_sound_money,4a_tariffs,4b_regulatory_trade_barriers,4c_black_market,4d_control_movement_capital_ppl,4_trade,5a_credit_market_reg,5b_labor_market_reg,5c_business_reg,5_regulation,id
0,1970,Albania,1.97,8.232353,7.509902,8.0,8.0,7.935564,2.668222,3.145462,4.512228,8.333333,4.166667,4.387444,6.485287,6.933500,6.215401,0.948718,5.071814,8.986454,9.484575,9.743600,10.0,9.553657,8.963556,7.489905,10.00000,6.406138,8.214900,7.098562,6.916278,6.705863,6.906901,1970Albania
1,1970,Algeria,2.47,2.150000,7.817129,0.0,4.5,3.616782,4.186704,4.327113,4.689952,4.166667,5.000000,4.507538,6.626692,6.136845,6.737383,0.820513,4.690743,6.955962,8.339152,8.720460,5.0,7.253894,6.872533,2.481294,5.56391,1.590362,4.127025,5.100509,5.029513,5.676956,5.268992,1970Algeria
2,1970,Angola,2.66,7.600000,8.886739,0.0,9.5,6.496685,1.843129,1.974566,2.512364,3.333333,4.166667,2.302200,5.455882,3.016104,4.291197,0.846154,2.963635,9.385679,4.986742,3.054000,5.0,5.606605,6.989244,2.024949,10.00000,2.044823,5.264754,7.064905,4.560325,4.930271,5.518500,1970Angola
3,1970,Argentina,2.71,5.335294,6.048930,6.0,4.0,5.346056,3.689690,2.930563,4.255995,7.500000,3.333333,3.632872,6.857195,3.385786,4.133832,0.769231,3.904030,5.233264,5.224079,2.000000,10.0,5.614336,6.421600,4.811105,0.00000,4.697482,3.982547,5.419820,5.151405,5.535831,5.369019,1970Argentina
4,1970,Armenia,2.74,7.264706,7.748532,8.0,5.0,7.003310,3.867379,4.197569,5.664317,5.833333,5.000000,5.196693,9.801963,5.714028,7.007654,1.000000,5.809215,9.078264,9.263496,9.746000,10.0,9.521940,8.547556,7.194410,10.00000,6.830998,8.143241,9.102046,6.234630,6.797530,7.378069,1970Armenia
5,1970,Australia,2.75,4.511765,6.664948,10.0,5.0,6.544178,8.766929,5.610653,7.998304,10.000000,9.166667,6.025331,8.205655,8.707445,6.790183,1.000000,7.919019,8.564838,9.500704,9.744620,10.0,9.452540,8.820000,7.427103,10.00000,3.859781,7.526721,9.576358,7.698589,8.121631,8.465526,1970Australia
6,1970,Austria,2.83,3.744118,3.101062,10.0,3.5,5.086295,7.650855,5.482200,8.092565,10.000000,10.000000,6.220993,8.100494,8.501867,7.194909,1.000000,7.915987,8.033869,9.875451,9.822000,10.0,9.432830,8.266133,8.254011,10.00000,5.906786,8.106733,9.129015,5.565118,7.419870,7.371334,1970Austria
7,1970,Azerbaijan,2.84,6.320588,7.881208,0.0,6.5,5.175449,5.676886,5.861119,6.321836,5.000000,5.833333,7.252962,9.830312,6.170495,7.280192,0.717949,5.652732,9.240398,6.649350,9.164200,5.0,7.513487,7.973778,6.799052,10.00000,4.093268,7.216525,7.467778,6.076782,7.080078,6.874880,1970Azerbaijan
8,1970,Bahamas,2.87,7.085294,9.040503,7.0,10.0,8.281449,4.698028,6.700000,5.322493,10.000000,6.666667,4.699257,6.149350,5.321382,6.059572,0.945946,6.658107,8.654101,8.975514,9.930800,0.0,6.890104,3.253067,8.105925,10.00000,5.384615,6.685902,8.622733,8.361606,7.523034,8.169125,1970Bahamas
9,1970,Bahrain,2.88,3.688235,9.581704,10.0,10.0,8.317485,6.822563,6.231401,7.435713,5.000000,7.500000,4.785814,8.855315,7.764706,7.287250,0.536585,5.265602,9.961596,8.215418,9.440400,10.0,9.404353,8.523200,6.295595,10.00000,4.794819,7.403403,7.609167,6.619418,8.018774,7.415786,1970Bahrain


In [32]:
data.groupby('countries')['ECONOMIC FREEDOM'].nunique()

countries
Albania                   19
Algeria                   19
Angola                    12
Argentina                 19
Armenia                   13
Australia                 19
Austria                   19
Azerbaijan                13
Bahamas                   19
Bahrain                   19
Bangladesh                19
Barbados                  19
Belarus                    1
Belgium                   19
Belize                    19
Benin                     19
Bhutan                     4
Bolivia                   19
Bosnia and Herzegovina    12
Botswana                  19
Brazil                    19
Brunei Darussalam          7
Bulgaria                  19
Burkina Faso              12
Burundi                   19
Cambodia                   7
Cameroon                  19
Canada                    19
Cape Verde                 7
Central Afr. Rep.         19
                          ..
Slovenia                  18
South Africa              18
Spain                     18
Sri 

In [0]:
features = ['1a_government_consumption', '1c_gov_enterprises', '1d_top_marg_tax_rate', '2a_judicial_independence', '3c_inflation',
'3d_freedom_own_foreign_currency', '4a_tariffs', '4c_black_market', '4d_control_movement_capital_ppl', '5a_credit_market_reg', '5b_labor_market_reg']

X = data[features]
y = data['ECONOMIC FREEDOM']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

print(linreg.intercept_)
print(linreg.coef_)
zip(features, linreg.coef_)

7.924269157407684
[ 0.05703258 -0.0130662  -0.06706412  0.09573019 -0.11494083  0.00819058
 -0.04101585  0.01193928 -0.00997308 -0.01845185 -0.08211089]


In [15]:
y_pred = linreg.predict(X_test)
y_pred

array([6.30649534, 6.55161059, 6.18039534, 5.99189797, 6.04517358,
       5.84085662, 6.15034634, 6.05204184, 6.67707085, 6.54378171,
       6.24928083, 6.54047609, 6.4412333 , 6.75691471, 6.4763778 ,
       6.21087032, 6.39621419, 6.38668749, 6.62737236, 7.2447198 ,
       6.76103363, 6.38410285, 6.37858311, 6.41341899, 6.08479579,
       6.1763282 , 5.89744606, 6.54409264, 6.63542791, 6.43682094,
       6.75115715, 6.41613059, 6.46791451, 6.63538579, 6.89841723,
       6.2775079 , 6.46690488, 6.2905052 , 6.28634193, 6.21849655,
       6.07878381, 6.20136406, 6.55616899, 5.99176697, 6.32154677,
       6.66142819, 6.4975586 , 6.56532657, 5.84249719, 6.3788932 ,
       7.33062246, 6.99509136, 6.40867508, 6.10064105, 6.10930385,
       6.64241595, 6.63155957, 6.33205321, 6.58511486, 6.25428088,
       6.51885422, 6.13401367, 6.47989519, 6.52941401, 6.17573876,
       6.96498235, 6.84658258, 6.61565194, 6.20403205, 6.70474863,
       6.256361  , 6.71622741, 6.58687611, 6.69990181, 6.35223

In [18]:
print(metrics.mean_absolute_error(y_test, y_pred))
print(metrics.mean_squared_error(y_true, y_pred))
#print(np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

0.7970679676255386


NameError: ignored